# Serving Models trained on Databricks with Azure ML

(install the required libraries beforehand)

In [ ]:
import requests
import json
import azureml
import mlflow.azureml
import numpy as np
import pandas as pd


from azureml.core.webservice import AciWebservice, AksWebservice, Webservice
from sklearn import datasets
from azureml.core.compute import AksCompute, ComputeTarget

## Create or load an Azure ML Workspace

azureml.core.Workspace.create()

## Get the trained model

model_uri = "dbfs:/databricks/mlflow/173665864247289/a60e5f4bcf9247f79b2ea985534229a6/artifacts/lrModelSK"

## Build an Azure Container image for the trained model

In [ ]:
model_image, azure_model = mlflow.azureml.build_image(model_uri=model_uri, 
                                                      workspace=workspace, 
                                                      model_name="cph-model",
                                                      image_name="cph-model-container-image",
                                                      description="skl for scoring machines",
                                                      synchronous=False)

In [ ]:
model_image.wait_for_creation(show_output=True)

## Deploy the model to "dev" using the model's container image
### Create an ACI webservice deployment

In [ ]:
dev_webservice_name = "cph-model-dev"
dev_webservice_deployment_config = AciWebservice.deploy_configuration()
dev_webservice = Webservice.deploy_from_image(name=dev_webservice_name, 
                                              image=model_image, 
                                              deployment_config=dev_webservice_deployment_config, 
                                              workspace=workspace)

In [ ]:
dev_webservice.wait_for_deployment()

### Query the deployed model in dev
(sample model trained with the telemetry data)

#### Load the sample input vector from the training data

In [ ]:
sparkDF = spark.read.format("delta").load("/databricks_workshop/validate_clean").limit(20)

train_x = sparkDF.toPandas()
sample = train_x.iloc[:1, 1:]
sample_json = sample.to_json(orient="split")
query_input = list(sample.as_matrix().flatten())

#### Evaluate the sample input by sending an HTTP request

In [ ]:
def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=inputs, headers=headers)
  print("Response: {}".format(response.text))
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

## Deploy the model to "prod" using the model's container image

### Create a new AKS cluster

# Use the default configuration (you can also provide parameters to customize this)
prov_config = AksCompute.provisioning_configuration()

aks_cluster_name = "cph-prod" 
# Create the cluster
aks_target = ComputeTarget.create(workspace = workspace, 
                                  name = aks_cluster_name, 
                                  provisioning_configuration = prov_config)

# Wait for the create process to complete
aks_target.wait_for_completion(show_output = True)
print(aks_target.provisioning_state)
print(aks_target.provisioning_errors)

### Deploy to the model's image to the specified AKS cluster

# Set configuration and service name
prod_webservice_name = "cph-prod"
prod_webservice_deployment_config = AksWebservice.deploy_configuration()

# Deploy from image
prod_webservice = Webservice.deploy_from_image(workspace = workspace, 
                                               name = prod_webservice_name,
                                               image = model_image,
                                               deployment_config = prod_webservice_deployment_config,
                                               deployment_target = aks_target)

In [ ]:
prod_webservice.wait_for_deployment(show_output = True)

### Query the deployed model in prod
(sample model trained with the telemetry data)

#### Load the sample input vector from the training data

In [ ]:
sparkDF = spark.read.format("delta").load("/databricks_workshop/validate_clean").limit(20)

train_x = sparkDF.toPandas()
sample = train_x.iloc[:1, 1:]
sample_json = sample.to_json(orient="split")
query_input = list(sample.as_matrix().flatten())

#### Evaluate the sample input by sending an HTTP request

In [ ]:
def query_endpoint_example(scoring_uri, inputs, service_key=None):
  headers = {
    "Content-Type": "application/json",
  }
  if service_key is not None:
    headers["Authorization"] = "Bearer {service_key}".format(service_key=service_key)
    
  print("Sending batch prediction request with inputs: {}".format(inputs))
  response = requests.post(scoring_uri, data=inputs, headers=headers)
  preds = json.loads(response.text)
  print("Received response: {}".format(preds))
  return preds

In [ ]:
prod_scoring_uri = prod_webservice.scoring_uri
prod_service_key = prod_webservice.get_keys()[0] if len(prod_webservice.get_keys()) > 0 else None

In [ ]:
prod_prediction = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=sample_json)

## Update the production deployment

### Train a new model (take a new model for example)

In [ ]:
model_uri_update = "dbfs:/databricks/mlflow/1622123275403635/e71c7cf2fabd487baf88e018ad282db1/artifacts/lrModelSK"

In [ ]:
model_image_updated, azure_model_updated = mlflow.azureml.build_image(model_uri=model_uri_update,
                                                                      workspace=workspace, 
                                                                      model_name="wine-rating-model",
                                                                      image_name="wine-model-container-image",
                                                                      description="Sklearn ElasticNet image for rating wines", 
                                                                      tags={},
                                                                      synchronous=False)

In [ ]:
model_image_updated.wait_for_creation(show_output=True)

### Deploy the new model's image to the AKS

In [ ]:
prod_webservic  e.update(image=model_image_updated)

In [ ]:
prod_prediction_updated = query_endpoint_example(scoring_uri=prod_scoring_uri, service_key=prod_service_key, inputs=sample_json)